In [11]:
questions=["what are the guidelines for the relocation within India","what is the exit process","what are the guidelines for academic learning at tridents"]
from rag.create_embedding import ExtractCreateEmbeddings
file_path=r"C:\Users\manjuprasad.n\Downloads\India Employee Handbook - 2024.pdf"
embed = ExtractCreateEmbeddings(file_path)
embed_values= embed.extract_text_from_pdf()  
import numpy as np
embedded_vectors = []
for i in range(len(embed_values)):
    embedded_vectors.append(embed_values[i][1])

np_vectors = np.array(embedded_vectors)





PAth:::    C:\Users\manjuprasad.n\Downloads\India Employee Handbook - 2024.pdf
Extracted text from page 1
Extracted text from page 2
Extracted text from page 3
Extracted text from page 4
Extracted text from page 5
Extracted text from page 6
Extracted text from page 7
Extracted text from page 8
Extracted text from page 9
Extracted text from page 10
Extracted text from page 11
Extracted text from page 12
Extracted text from page 13
Extracted text from page 14
Extracted text from page 15
Extracted text from page 16
Extracted text from page 17
Extracted text from page 18
Extracted text from page 19
Extracted text from page 20
Extracted text from page 21
Extracted text from page 22
Extracted text from page 23
Extracted text from page 24
Extracted text from page 25
Extracted text from page 26
Extracted text from page 27
Extracted text from page 28
Extracted text from page 29
Extracted text from page 30
Extracted text from page 31
Extracted text from page 32
Extracted text from page 33
Extrac

In [41]:
questions=['Types of expenses covered for relocation pplicy within India','What are half yearly awards','What is the criteria for PAT on the back','What is sexual  harasement','Memebers and contact details of ICC','Policy relatd to internet usage','Return of assets pplicy','Category -1 of Tredenecy accedamy of lots of Learning','Category -4 of Tredenecy accedamy of lots of Learning','Category -5 of Tredenecy accedamy of lots of Learning']

In [42]:
q_embed=[]
from rag.create_embedding import embed_text
for q in questions:
    target_vector = np.array(embed_text(q))
    q_embed.append(target_vector)

In [44]:
len(q_embed)

10

In [53]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from utils.config import cos_top_n, knn_top_n


class SimilaritySearch:

    def __init__(self):
        self.top_n = cos_top_n
        # Initialize the NearestNeighbors model with cosine similarity
        self.knn_model = NearestNeighbors(n_neighbors=knn_top_n, metric='manhattan')

    # Function to perform KNN similarity search using cosine similarity
    def knn_similarity_search(self, target_vector, vectors):
        # Fit the model on the dataset of vectors
        self.knn_model.fit(vectors)
        
        # Reshape target vector to 2D (as required by scikit-learn)
        target_vector = np.array(target_vector).reshape(1, -1)
        
        # Perform the KNN search
        distances, indices = self.knn_model.kneighbors(target_vector)
        
        # Return the indices of the nearest vectors and their distances
        return indices.flatten(), distances.flatten()


    # Function to compute cosine similarity and return top N most similar vectors
    def cosine_similarity_search(self, target_vector, vectors):
        # Reshape target_vector to 2D since cosine_similarity expects 2D arrays
        target_vector = np.array(target_vector).reshape(1, -1)
        
        # Compute cosine similarities between the target vector and all other vectors
        similarities = cosine_similarity(vectors, target_vector)
        
        # Flatten the result array
        similarities = similarities.flatten()

        # Get indices of top N most similar vectors (in descending order of similarity)
        top_n_indices = similarities.argsort()[::-1][:self.top_n]
        
        # Get corresponding similarity scores
        top_n_similarities = similarities[top_n_indices]
        return top_n_indices, top_n_similarities
    


    def search(self, target_vector, vectors):
        
        knn_top_indices, knn_top_distances = self.knn_similarity_search(target_vector, vectors)
        cos_top_indices, cos_top_similarities = self.cosine_similarity_search(target_vector, vectors)

       
        
        return cos_top_indices,cos_top_similarities,knn_top_indices,knn_top_distances


In [54]:
formatted_output = []
ss=SimilaritySearch()
for qr in range(len(questions)):  
    cos_top_indices, cos_top_similarities, knn_top_indices, knn_top_distances = ss.search(target_vector=q_embed[qr], vectors=np_vectors)
    output = [questions[qr]] 
    for idx, similarity in zip(cos_top_indices, cos_top_similarities):
        output.append(f"{idx},{similarity}")
    for idx, distance in zip(knn_top_indices, knn_top_distances):
        output.append(f"{idx},{distance}")
    formatted_output.append(output)

print(formatted_output)


[['Types of expenses covered for relocation pplicy within India', '24,0.73167514891693', '26,0.7234795860880974', '25,0.7206877359265444', '26,16.31082082206285', '24,16.39321315582947', '25,16.624978481821927'], ['What are half yearly awards', '33,0.6012876925602934', '34,0.5745539086034994', '32,0.5488804145321222', '33,19.63542295501611', '34,20.477088664374605', '32,21.018170781212348'], ['What is the criteria for PAT on the back', '36,0.4221254111180226', '58,0.38948894705687287', '33,0.38037380928989345', '36,23.795754771310385', '58,24.214489592352038', '33,24.60175469755268'], ['What is sexual  harasement', '48,0.656886157314919', '47,0.6556138661096768', '46,0.6120114280001866', '47,18.297893255847157', '48,18.3714738369963', '46,19.334288427149204'], ['Memebers and contact details of ICC', '50,0.5180903247334876', '49,0.49397524510879826', '51,0.42008281396055897', '50,21.960020107522723', '49,22.234579990414204', '54,23.655519608524628'], ['Policy relatd to internet usage', 

In [55]:
import pandas as pd
# df_=pd.DataFrame(formatted_output)
columns = ['question', 'cos1', 'cos2', 'cos3', 'knn1', 'knn2', 'knn3']

# Create the DataFrame
df = pd.DataFrame(formatted_output, columns=columns)

# Display the DataFrame
display(df)

,question,cos1,cos2,cos3,knn1,knn2,knn3
0,Types of expenses covered for relocation pplic...,"24,0.73167514891693","26,0.7234795860880974","25,0.7206877359265444","26,16.31082082206285","24,16.39321315582947","25,16.624978481821927"
1,What are half yearly awards,"33,0.6012876925602934","34,0.5745539086034994","32,0.5488804145321222","33,19.63542295501611","34,20.477088664374605","32,21.018170781212348"
2,What is the criteria for PAT on the back,"36,0.4221254111180226","58,0.38948894705687287","33,0.38037380928989345","36,23.795754771310385","58,24.214489592352038","33,24.60175469755268"
3,What is sexual harasement,"48,0.656886157314919","47,0.6556138661096768","46,0.6120114280001866","47,18.297893255847157","48,18.3714738369963","46,19.334288427149204"
4,Memebers and contact details of ICC,"50,0.5180903247334876","49,0.49397524510879826","51,0.42008281396055897","50,21.960020107522723","49,22.234579990414204","54,23.655519608524628"
5,Policy relatd to internet usage,"60,0.6073482784573536","69,0.589736844478409","62,0.5831026740181822","60,19.768584935452964","69,19.988681321821787","62,20.05551515465686"
6,Return of assets pplicy,"74,0.5146504044852211","56,0.47058540171641006","58,0.4687237435630903","74,21.573567448399444","56,22.5445698742044","58,22.766111272800117"
7,Category -1 of Tredenecy accedamy of lots of L...,"30,0.5934275481729206","28,0.567482896736586","29,0.5467336141713287","30,20.021148398280275","28,20.91351215379109","29,21.040420193263344"
8,Category -4 of Tredenecy accedamy of lots of L...,"30,0.6067323334881924","28,0.5631720619363616","29,0.5398300672158118","30,19.836981748907874","28,20.89594665706545","29,21.13632673395159"
9,Category -5 of Tredenecy accedamy of lots of L...,"30,0.5969537409517347","28,0.5541485139406694","29,0.5348244575456846","30,19.892660828876615","28,21.006797655187256","29,21.197709010519247"


In [56]:
df.to_csv("itr.csv")